<a href="https://colab.research.google.com/github/KooTakin/academicpages.github.io/blob/master/%E2%80%9CMAFS5370_A2_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from copy import deepcopy

BOARD_ROW = 4
BOARD_COL = 4
WIN_COUNT = 4
ITER = 1000
C = 2 # Exploration constant

# Chess Board
class Board():

  def __init__(self, board=None):

    # Players
    self.player_1 = 'x'
    self.player_2 = 'o'
    self.empty_position = '.'

    # Positions
    self.position = {}

    # Reset
    self.reset_board()

    # Previous board state
    if board is not None:
      self.__dict__ = deepcopy(board.__dict__)

  # Reset the board
  def reset_board(self):

    for row in range(BOARD_ROW):
      for col in range(BOARD_COL):
        self.position[row, col] = self.empty_position


  def move(self, row, col):
    if random.random() <= 0.5:  # 50% chance to place at the chosen square
      return self.apply_move(row, col)
    else:
      return self.random_move(row, col)  # Otherwise, try a random move

  def apply_move(self, row, col):
      if self.position.get((row, col), None) == self.empty_position:
          board = Board(self)  # inherit from current position
          board.position[row, col] = self.player_1  # Make move
          (board.player_1, board.player_2) = (board.player_2, board.player_1)
          return board
      else:
        board = Board(self)
        (board.player_1, board.player_2) = (board.player_2, board.player_1)
        return board  # Move forfeited if the spot is not empty

  def random_move(self, row, col):
      adjacent_positions = [
          (row-1, col-1), (row-1, col), (row-1, col+1),
          (row, col-1), (row, col+1),
          (row+1, col-1), (row+1, col), (row+1, col+1)
      ]
      valid_positions = [(r, c) for (r, c) in adjacent_positions if 0 <= r < BOARD_ROW and 0 <= c < BOARD_COL]
      valid_positions = [(r, c) for (r, c) in valid_positions if self.position.get((r, c), None) == self.empty_position]
      if valid_positions:
        chosen_row, chosen_col = random.choice(valid_positions)
        return self.apply_move(chosen_row, chosen_col)
      else:
        (board.player_1, board.player_2) = (board.player_2, board.player_1)
        return board  # Move forfeited if all random choices are invalid
  # Check drawing state
  def is_draw(self):

    for row, col in self.position:
      if self.position[row, col] == self.empty_position:
        return False

    return True

  # Check winning state

  def is_win(self):

    # Check vertical positions
    for col in range(BOARD_COL):
      winning_sequence = []
      for row in range(BOARD_ROW):
        if self.position[row, col] == self.player_2: # Notice that we have switched the order when moving
          winning_sequence.append(self.player_2)
        else:
          if len(winning_sequence) != 0:
            break
        if len(winning_sequence) == WIN_COUNT:
          return True

    # Check horizontal positions
    for row in range(BOARD_COL):
      winning_sequence = []
      for col in range(BOARD_ROW):
        if self.position[row, col] == self.player_2: # Notice that we have switched the order when moving
          winning_sequence.append('1')
        else:
          if len(winning_sequence) != 0:
            break
        if len(winning_sequence) == WIN_COUNT:
          return True

    # Check left diagonal positions
    winning_sequence = []
    for row in range(BOARD_ROW):
      col = row
      if self.position[row, col] == self.player_2: # Notice that we have switched the order when moving
        winning_sequence.append('1')
      else:
        if winning_sequence != 0:
          break

      if len(winning_sequence) == WIN_COUNT:
        return True

    # Check right diagonal positions
    winning_sequence = []
    for row in range(BOARD_ROW):
      col = BOARD_ROW - row - 1
      if self.position[row, col] == self.player_2: # Notice that we have switched the order when moving
        winning_sequence.append('1')
      else:
        if winning_sequence != 0:
          break

      if len(winning_sequence) == WIN_COUNT:
        return True

    return False

  # Legal moves
  def action_space(self):

    actions = []

    for row in range(BOARD_ROW):
      for col in range(BOARD_COL):
        # make sure that current square is empty
        actions.append(self.apply_move(row, col))

    return actions



  # Main game loop
  def game_loop(self):

    print('\n  Super Tic Tac Toc')
    print('  Enter the move like: 1,1 ([row], [col]) (The first index is 1.)')
    print('\n  Super Tic Tac Toc')
    print('  Enter the move like: 1,1 ([row], [col]) (The first index is 1.)')
    print("  After a player chooses an empty square, there is only ½ chance that his nought or cross is placed at the chosen square.")
    print("  If the player's choice is not accepted, the player's move is selected randomly with probability 1/16 by the computer from the 8 random squares adjacent to the chosen one, with the boundaries ignored.")
    print("  If the random choice is occupied or outside of the board, the player's move is forfeited.")
    print("  For example, if the chosen square is at the corner, with probability 5/16 the randomly selected square is outside of the board.")
    print('  Type "quit" to quit the game.')

    print(self)

    # AI agent
    mcts = MCTS()

    while True:

      user_input = input('> ')
      if user_input == 'quit':
        break

      if user_input == '':
        continue

      try:

        row = int(user_input.split(',')[0]) - 1
        col = int(user_input.split(',')[1]) - 1

        self = self.move(row, col)
        print(self)

        if self.is_win():
          print('Player "%s"' % self.player_2, "has won the game! Congratulation!")
          break
        elif self.is_draw():
          print("Game is drawn.")
          break

        # AI turn
        try:
          best_move = mcts.search(self)
          print(best_move)
          self = best_move.board

        except Exception as e:
          print(e)

        print(self)


        # Check win or draw
        if self.is_win():
          print('Player "%s"' % self.player_2, "has won the game! Congratulation!")
          break
        elif self.is_draw():
          print("Game is drawn.")
          break

      except:
        print("Invalid input!")
        continue

  # Print the board
  def __str__(self):

    board_str = ''

    for row in range(BOARD_ROW):
      for col in range(BOARD_COL):
        board_str += " %s" % self.position[row, col]
      board_str += '\n'

    if self.player_1 == 'x':
      board_str = '\n ---------------- \n "x" to move: \n----------------\n\n' + board_str

    elif self.player_1 == 'o':
      board_str = '\n ---------------- \n "o" to move: \n----------------\n\n' + board_str

    return board_str

if __name__ == '__main__':
  # Create board
  board = Board()
  board.game_loop()




  Super Tic Tac Toc
  Enter the move like: 1,1 ([row], [col]) (The first index is 1.)

  Super Tic Tac Toc
  Enter the move like: 1,1 ([row], [col]) (The first index is 1.)
  After a player chooses an empty square, there is only ½ chance that his nought or cross is placed at the chosen square.
  If the player's choice is not accepted, the player's move is selected randomly with probability 1/16 by the computer from the 8 random squares adjacent to the chosen one, with the boundaries ignored.
  If the random choice is occupied or outside of the board, the player's move is forfeited.
  For example, if the chosen square is at the corner, with probability 5/16 the randomly selected square is outside of the board.
  Type "quit" to quit the game.

 ---------------- 
 "x" to move: 
----------------

 . . . .
 . . . .
 . . . .
 . . . .

> 3,3

 ---------------- 
 "o" to move: 
----------------

 . . . .
 . . x .
 . . . .
 . . . .



In [1]:
# MCTS

import math
import random

# tree node
class TreeNode():

  def __init__(self, board, parent):

    self.board = board
    # Terminal node
    if self.board.is_win() or self.board.is_draw():
      self.is_terminal = True
    else:
      self.is_terminal = False

    # Check if the set is fully expanded
    self.is_fully_expanded = self.is_terminal

    self.parent = parent

    # The number of node that has visited
    self.visits = 0

    # The total score of the node
    self.score = 0

    self.children = {}

class MCTS():

  # Search for the best move
  def search(self, initial_state):
    self.root = TreeNode(initial_state, None)

    for iteration in range(ITER):
      node = self.select(self.root)
      # Score the node
      score = self.rollout(node.board)
      # Backpropogate result
      self.backpropogate(node, score)
    # Pick up the best move
    print(self.get_best_move(self.root, 0))
    return self.get_best_move(self.root, 0)

  # Select the node
  def select(self, node):

    while not node.is_terminal:
      # fully expanded
      if node.is_fully_expanded:
        node = self.get_best_move(node, C)
        if node is None:
          print("Warning: get_best_move return None!")

      # not fully expanded
      else:
        new_node = self.expand(node)
        if new_node is None:
          print("Warning: no valid expansion.")
        return self.expand(node)

    return node

  def expand(self, node):
    # Generate legal states
    states = node.board.action_space()

    for state in states:
      # make sure that current state is not present in child nodes
      if str(state.position) not in node.children:
        new_node = TreeNode(state, node)
        node.children[str(state.position)] = new_node # add the child node to parent's node children list

        # Check if (parent) fully expanded
        if len(states) == len(node.children):
          node.is_fully_expanded = True

        return new_node


  # Simulate the game by making moves until the end of the game
  def rollout(self, board):

    while not board.is_win():
      try:
        board = random.choice(board.action_space())

      except:
        return 0 # Draw score

    if board.player_2 == 'x':
      return 1
    elif board.player_2 == 'o':
      return -1

  # Backpropogate the number of visits and score up to the root node
  def backpropogate(self, node, score):

    # Update node visit count
    while node is not None:
      node.visits += 1
      node.score += score
      node = node.parent

  # Find the best move based on UCB1/UCT formula
  def get_best_move(self, node, eps):

    best_score = float("-inf")
    best_moves = []

    # loop for child nodes
    for child_node in node.children.values():
      if child_node.board.player_2 == 'x':
        current_player = 1
      elif child_node.board.player_2 == 'o':
        current_player = -1

      # Move score
      move_score = current_player * child_node.score / (child_node.visits+0.00000000000001) + eps * math.sqrt(math.log(node.visits) / (child_node.visits+0.0000000000001))

      if move_score > best_score:
        best_score = move_score
        best_moves = [child_node]

      elif move_score == best_score:
        best_moves.append(child_node)

    return random.choice(best_moves)




MCTS keeps the gain tree in the memory. We need the backpropogation

Trick: Switch the order of Player 1 and Player 2
